#### Callahan's Modification for Ionosonde Retrieval

# Lowell Digisonde Reader
This notebook is a tool to read and plot data from the Lowell Digisonde network: https://www.digisonde.com/

Follow the Rules of the Road! https://giro.uml.edu/didbase/RulesOfTheRoad.html

In [1]:
import urllib
import pandas as pd
import plotly.express as px
import wget
import os                                      # for making sure we have a directory to write data to


# Generate output directories:
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('plots'):
    os.makedirs('plots')

## Read Station List


In [2]:
original_stations = pd.read_csv('DIDBASE List.csv')
original_stations = original_stations.set_index('URSI')
# original_stations['LONG'] = original_stations['LONG'] - 360
display(original_stations.head())
locations = ['EIELSON','WALLOPS IS','ALPENA','IDAHO NATIONAL LAB','MELROSE','BOULDER','AUSTIN'] #'KENT IS',

,#,STATION NAME,LAT,LONG
URSI,,,,
AH223,1,AHMEDABAD,23.00,72.50
AL945,2,ALPENA,45.07,276.44
AN438,3,ANYANG,37.39,126.95
AS00Q,4,ASCENSION ISLAND,-7.95,345.60
AT138,5,ATHENS,38.00,23.50


## Map stations

In [3]:
# Map nodes:
scatter_df = original_stations.loc[original_stations["STATION NAME"].isin(locations),:]
scatter_df['LONG'] = scatter_df['LONG'] + 360
display(scatter_df)
fig = px.scatter_geo(scatter_df, "LAT", "LONG",
#                      color="Status", # which column to use to set the color of markers
                     hover_name=scatter_df["STATION NAME"].tolist(), # column added to hover information
                     hover_data=[scatter_df], #.index.tolist(),
                     )
fig.update_layout(title="Stations")
fig.show()
fig.write_html("ionosonde_map.html", include_plotlyjs="cdn")

C:\cygwin64\tmp\ipykernel_35300\49338526.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scatter_df['LONG'] = scatter_df['LONG'] + 360


,#,STATION NAME,LAT,LONG
URSI,,,,
AL945,2,ALPENA,45.07,636.44
AU930,6,AUSTIN,30.40,622.30
BC840,11,BOULDER,40.00,614.70
EI764,29,EIELSON,64.66,572.93
AC843,46,IDAHO NATIONAL LAB,43.81,607.32
IF843,47,IDAHO NATIONAL LAB,43.81,607.32
ME929,68,MELROSE,29.71,638.00
WP937,116,WALLOPS IS,37.90,644.50


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## Functions to read data: 

### Construct URL
Example: https://lgdc.uml.edu/common/DIDBGetValues?ursiCode=BC840&charName=hmE&fromDate=2020.01.29&toDate=2020.01.30

In [ ]:
# url = "https://lgdc.uml.edu/common/DIDBGetValues?ursiCode=BC840&charName=hmE&fromDate=2020.01.29&toDate=2020.01.30"

import requests

# Parameters:
    # f0F2 f0F1 MD (MUF(D) /foF2) fmin f0ES fminF fminE foE fxI hF hF2 hE hEs hmE yE QF QE FF FE
    # hmF2 hmF1 zhalfNm finEs yF2 yF2 TEC scaleF2 (scale height at F2 peak) B0 B1 D1 f0Ea
    # hEa foP hP fbEs TypeEs

def urlmaker(URSIcode, parameter, fromDate, toDate):
    URL = 'http://lgdc.uml.edu/common/DIDBGetValues' + \
      '?ursiCode=' + URSIcode + \
      '&charName=' + parameter + \
      '&fromDate=' + fromDate + \
      '&toDate='   + toDate
    return URL


url = urlmaker(URSIcode = 'BC840', parameter = 'hmE', fromDate = '2020.01.29', toDate = '2020.01.30')
url

In [ ]:
def didread(url):
    file = urllib.request.urlopen(url)
#     print(file)
    db_id = []                                                     # commented lines
    ego = []                                                    # get it? as opposed to "id"? (I'm tired.)
    for line in file:
#         print(line)
        decoded_line = line.decode("utf-8")
#         non_utf = line.decode()
#         print(non_utf)
        if decoded_line.startswith("#"):
    #         print(decoded_line)
            db_id.append(decoded_line)
        else:
            ego.append(decoded_line.split())
#     print(db_id)                                                          # print header information

    headers = db_id[len(db_id)-1]                                     # Pull headers from url file
#     print(headers)
    headers.split()                                             
    df = pd.DataFrame(ego)                                      # Create dataframe
    df.columns = headers.split()
    df=df.rename(columns = {'#Time':'Time'})                    # Fix column name
    df['Time'] = pd.to_datetime(df['Time'])                     # Cast timestamp to datetime
    return df

### Read URL data

### Plot data from multiple ionosondes for a given date and parameter:

First, set your parameters of interest in this cell:

In [ ]:
# This stuff's not going to change from station to station:
parameters = ['hmF2','hmE','foF2','foE']
fromDate = '2017.01.01' 
toDate = '2017.12.31'
cadence = pd.Timedelta(hours=1)
locations = ['EIELSON','WALLOPS IS','ALPENA','IDAHO NATIONAL LAB','MELROSE','BOULDER','AUSTIN'] #'KENT IS',
debug = False

if isinstance(parameters, str): parameters = [parameters]

all_stations = original_stations.index.unique().tolist()
# Let's confine our search to just a couple of stations. Comment this line out if you want to see ALL of them. 
stations = original_stations.loc[original_stations['STATION NAME'].isin(locations),:]
display(stations)
display(original_stations.loc[original_stations['STATION NAME'].isin(locations),'LONG'])

In [ ]:
# How to generate df for a single station and set of parameters:
# df = didread(urlmaker(URSIcode = 'BC840', parameter = 'hmE', fromDate = '2020.01.29', toDate = '2020.01.30'))

debug=False

# Preallocation:
def api_parameter(parameter):
    df = didread(urlmaker(URSIcode = 'AT138', parameter = parameter, fromDate = fromDate, toDate = toDate))
    df = df.drop(columns = ['CS', 'QD'])
    df['Time']= pd.to_datetime(df['Time']) # cast to datetime
    df = df.set_index('Time')
#     pd.DataFrame.resample(df, cadence)
    return df

param_add = lambda x, parameter : f'{parameter} {x}'

if isinstance(parameters, list):
    df = api_parameter(parameters[0])
    if len(parameters) > 1:
        for parameter in parameters[1:]:
            temp_df = api_parameter(parameter)
            temp_df.columns = [param_add(col, parameter) for col in temp_df.columns]
            if debug: display(temp_df.head(3))
            df = pd.merge(df, temp_df, left_index=True, right_index=True, how='outer')
else:
    raise AttributeError
    
if debug: display(df)

def check_available(stations, parameters):
    if not isinstance(stations, (list, tuple)):
        stations = list(stations)
    if isinstance(parameters, str):
        parameters = [parameters]
    iterations = 0

    total_iterations = len(all_stations) * len(parameters)
    valid_args = list()
    for station in sorted(all_stations):
        name = original_stations.loc[station,'STATION NAME']
        for parameter in parameters:
            print(f'Iteration {iterations}/{total_iterations}', end='\r')
            iterations += 1
            try:
                url_ret = urlmaker(URSIcode=station, parameter=parameter, fromDate=fromDate, toDate=toDate)
                df1 = didread(url_ret)
                print(f'{name}::{station} {parameter} Successful')
            except Exception as e:
                print(f'{name}::{station} {parameter} Unsuccessful')
                continue
            else:
                valid_args.append(f'{station} {parameter}')
    return valid_args

# valid_args = check_available(all_stations, parameters[0])
# Iterating through all the ionosondes: 
for station in stations.index:
    print(station)
    if isinstance(parameters, str): parameters = [parameters]
    for parameter in parameters:
    #     try:
        try:
            temp_df = didread(urlmaker(URSIcode=station, parameter=parameter, fromDate=fromDate, toDate=toDate))
#             display(temp_df.head())
        except Exception as e:
            print(e)
            continue
        s_row = stations.loc[stations.index==station,:]
#         display(foo)
        temp_df = temp_df.rename(columns = {str(parameter):s_row.iloc[0, 1]})
        temp_df = temp_df.drop(columns = ['CS', 'QD'])

        temp_df['Time']= pd.to_datetime(temp_df['Time'])
        temp_df = temp_df.set_index('Time')
#         df1 = df1.resample(df1, cadence)

        if debug: display(temp_df)
        temp_df.columns = [param_add(col, parameter) for col in temp_df.columns]
        if debug: print(temp_df.columns)
        print('Merging data...' + station +": " + s_row.iloc[0, 1])
        df = pd.merge(df, temp_df, on = 'Time', how='outer')  

#     except:
#         print('Exception Occurred')
#         continue
if debug: display(df)

def col_in_locations(col):
    for location in locations:
        if col.endswith(location):
            return True
    return False

cols = [col for col in df.columns if col_in_locations(col)]
df = df.loc[:,cols].sort_index()
display(df)
df_name = f'data/{pd.to_datetime("today").strftime("%H:%M")}.csv'
df.to_csv(df_name)

In [ ]:
# from datetime import datetime

# lon_dict = { # use original station data
#     'BOULDER' : -105.2705,
#     'AUSTIN' : -97.7431,
# }

# # Tuscaloosa Example
# utc = datetime.now()
# lon = -87.57
# dhr = lon / 15.
# loc = utc + pd.Timedelta(hours=dhr)

mst = lambda utc, lon : utc + pd.Timedelta(hours=lon / 15)

In [ ]:
df = pd.read_csv(df_name, index_col=0)
df.index = pd.to_datetime(df.index)
    
print(df.head(1))
print('\nHere is the ' + str(parameter) + ' data you requested:')



group_12hour = lambda x : x.groupby(pd.Grouper(level=0, freq='12H')).mean()
# df = df.dropna()


original_index = df.index.copy()
# df = group_12hour(df)

for location in locations:
    print(location)
    code = original_stations.loc[original_stations['STATION NAME'] == location].index.to_numpy()[0]
    longitude = original_stations.loc[code,'LONG']
    df[f'{location} MST'] = pd.to_datetime([mst(time, longitude) for time in df.index])
display(df.columns)

In [ ]:
from functools import reduce

def group(df, locations):
    if isinstance(locations, str): locations = [locations]
    df_list = list()
    for location in locations:
        cols = [col for col in df.columns if location in col]
        index_col = f'{location} MST'
        loc_df = df.loc[:,cols].set_index(index_col)
        night_df = loc_df.between_time('8:00PM','4:00AM')
        night_df.columns = [col + ' Night' for col in night_df.columns]
        day_df = loc_df.between_time('8:00AM','4:00PM')
        day_df.columns = [col + ' Day' for col in day_df.columns]
        df_list.extend([night_df, day_df])
    m = lambda df1, df2: pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
    df = reduce(m, df_list)
    return df
grouped_df = group(df, locations)

In [ ]:
for col in grouped_df:
    display(grouped_df[col].dropna().sort_index())

In [ ]:
# df.index = df.index.shift(-6, freq='H')
# print(df.columns)

# df1 = df.iloc[0::2,:]
# df2 = df.iloc[1::2,:]
# df1.index = df1.index.shift(12, freq='H')

# new_df = pd.DataFrame()
# for location in locations:
#     new_df[f'{location} MST'] = df1.loc[:,f'{location} MST']
#     for parameter in parameters:
#         midnight_name = f'{parameter} {location} Night'
#         noon_name = f'{parameter} {location} Day'
#         try:
#             new_df[midnight_name] = df1.loc[:,f'{parameter} {location}']
#             new_df[noon_name] = df2.loc[:,f'{parameter} {location}']
#         except KeyError as ke:
#             print(ke)
#             continue
# display(new_df)

In [ ]:
import statsmodels.api as sm
from statsmodels.nonparametric.smoothers_lowess import lowess
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as pe
from matplotlib import rcParams
import seaborn as sns
import math
import numpy as np

In [ ]:
rcParams['font.size']      = 12
rcParams['font.weight']    = 'bold'
rcParams['axes.grid']      = True
rcParams['grid.linestyle'] = ':'
rcParams['figure.figsize'] = np.array([15, 8])
rcParams['axes.xmargin']   = 0
rcParams["figure.facecolor"] = 'white'
rcParams['axes.edgecolor'] = 'black'

In [ ]:
import math
plot_df = grouped_df.copy()
# plot_df = new_df
# plot_df.index.name = 'X'
# display(plot_df)

def cycle_colors(palette_str='Spectral', n_colors=8):
    desat1 = sns.color_palette(palette_str, n_colors)[::-1]
    desat2 = sns.color_palette(palette_str, n_colors, .4)[::-1]
#     palette = sns.color_palette(palette_str, n_colors)
    i = 0
    while(True):
        loc = int(i) % n_colors*2
        i += .5
        if i % 2 == 0:
            yield desat1[loc // 2]
        else:
            yield desat2[loc // 2]
            
def create_subplots(n, ncols=2, sharey=False):
    nrows = math.ceil(n / 2)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharey=sharey, sharex=True, figsize=(16,4*nrows), dpi=300)
    if not isinstance(axs, np.ndarray): axs = np.array(axs)
    for ax in axs.flatten():
        yield fig, ax
    
def resample_freq(series):
    series = series.copy()
    series.index = series.index.shift(freq='6H')
    series = series.groupby(pd.Grouper(level=0, freq='12H', origin='epoch')).mean()
    series = series.dropna()
    series.index = series.index.shift(freq='-6H')
    return series

def plot_individual(locations, parameters=['hmF2'], tods=['Night','Day'], sharey=False, freq='24H'):
#     matplotlib.style.use('seaborn')
    color_gen = cycle_colors(palette_str='Spectral', n_colors=len(locations)*2)
    ax_gen = create_subplots(len(locations)*len(parameters)*len(tods), ncols=len(tods), sharey=sharey)
    for location in locations:
        code = original_stations.loc[original_stations['STATION NAME'] == location].index.to_numpy()[0]
        lat = original_stations.loc[code,'LONG']
        lon = original_stations.loc[code,'LONG']
        for parameter in parameters:
            for tod in tods:
                no_data=False
                col = f'{parameter} {location} {tod}'
                try:
                    y = plot_df.loc[:,col]
                    y = y[~y.isnull()]
                except KeyError:
                    print(f'No MST data found for {col}')
                    no_data=True
                    continue

                y = resample_freq(y)
                X = pd.to_datetime(y.index)
#                 x_y_columns = [f'{location} MST',col]
#                 try:
#                     Xy = plot_df.loc[:,x_y_columns].dropna()
#                 except KeyError:
#                     print(f'No MST data found for {x_y_columns}')
#                     no_data=True
#                 else:
                y_hat = lowess(y.to_numpy(), X, frac=.15)
                    
                try:
                    fig, ax = next(ax_gen)
                except StopIteration:
                    fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(16,4), dpi=300, linewidth=10, edgecolor='black')
#                 plt.figure(figsize=(16,8))
#                 display(X)
                ax.grid(color='grey', linestyle='-.', linewidth=0.7)
                ax.set_facecolor((1, 1, 1))
                col = col.replace('Day', f'{lat_lon} Day').replace('Night', f'{lat_lon} Night')
                lat_lon = f'({lat},{lon})'
                try:
                    title = col + ' ' + X.min().strftime('%Y')
                except ValueError:
                    title = col
                ax.set_title(title, loc='left', fontname="DejaVu Sans", fontweight='bold')
                if not no_data:
                    ax.scatter(X, y, label='Raw Data', color=next(color_gen), edgecolor='black', linewidth=.5)
                    path_effects=[pe.Stroke(linewidth=7, foreground='k'), pe.Normal()]
                    ax.plot(X, y_hat[:,1], label='LOWESS', color=next(color_gen), linewidth=6, path_effects=path_effects)
                ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%b'))
                ax.legend(loc='lower left', shadow=True, frameon=True, facecolor='white')
        if parameter == 'hmF2':
            plt.ylim([100,500])
    fig.suptitle(parameters[0], fontweight='bold', fontsize=24)
#     plt.tight_layout()
    plt.show()
    return

print(parameter)
print(locations)
locationsA = locations[0:7]
locationsB = [location for location in locations if location not in locationsA]
for parameter in parameters:
    plot_individual(locations, parameters=[parameter], tods=['Night','Day'], sharey=True)
    print()
    print()

In [ ]:
# Generate an interactive plot
# df = df.sort_values(by="Time")  # This seems to break the plot for some reason.

# Convert to numeric so plotly express can plot the data correctly:
df[df.columns] = df[df.columns].apply(pd.to_numeric)

print(df.info())     # for debugging:


fig = px.line(df, title=str(parameter))


fig.update_layout(autotypenumbers='convert types')     # converting to fix plotting


fig.update_layout(
    title="Ionosonde Data: " + str(parameter),
    xaxis_title="Time (UTC)",
    yaxis_title=str(parameter),
    legend_title="Station",
    font=dict(
        family="Courier New, monospace",
        size=18,
#         color="RebeccaPurple"
    )
)

# Eliminate the gray background plotly express uses by default:
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

fig.show()
fig.write_html("plots/"+ str(parameter) + "_plot.html", include_plotlyjs="cdn")


In [ ]:
# # Comment in to write dataframe to CSV:
df.to_csv('data/' + str(parameter) + '.csv')